In [ ]:
import os

import tensorflow as tf
import sewar.full_ref
from tqdm import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt

import src.model
import src.selectioner
import src.utils
import src.vizualization

If you want to use different images in training, put them into `./images/contents` and `./images/styles` folders and change `style_image_file` and `content_image_file` variables to appropriate values.   

In [ ]:
style_image_file = "demo_picasso_music.jpg"
content_image_file = "demo_pablo_picasso.jpg"

BASE_RESULT_PATH = os.path.join("images", "results")
BASE_TRAINER_PATH = os.path.join("images", "trainers")

style_image_path = os.path.join("images", "styles", style_image_file)
content_image_path = os.path.join("images", "contents", content_image_file)

style_image = src.utils.tf_utils.load_img(style_image_path)
content_image = src.utils.tf_utils.load_img(content_image_path)


- Create 16 random `NSTImageTrainer`. Each of them is responsible for generating one image. All input images are the same, but because of differences in trainers initializations images generated by them should be slightly different. All trainers share the same VGG backbone model, so you don't have to worry about running out of the RAM.  
- Trainers are created from layers selectors, which means provided functions will pick layers that will capture style and content for you.  
- Selectors used in this code pick layers randomly with specified weights. You can modify them or write your own in [./src/utils/randomizers.py](./src/utils/randomizers.py) file.


In [ ]:
trainers= []
trainers_num = 16

for _ in tqdm(range(trainers_num)):
    trainer = src.model.NSTImageTrainer.from_layers_selectors(
            style_image,
            content_image,
            src.utils.randomizers.random_length_choices,
            src.utils.randomizers.normal_choice,
            trainer_kw = dict(total_variation_weight=120),
            style_layers_selector_kw = dict(min_output_elements_num=2, rel_loc=0.05, rel_scale=0.25), 
            content_layers_selector_kw = dict(rel_loc=0.5, rel_scale=0.26)
        )
    trainer.compile(tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1))
    trainers.append(trainer)

Job of `TraintersSelectioner` is to pick most interesting `trainers` for you. It has implemented methods that allow its to train trainers images, sort them by given criteria or remove the most "boring" ones.

In [ ]:
selectioner = src.selectioner.TraintersSelectioner(trainers)

You can provide callbacks that will be called after every trainer finish his training. There are two predefined callbacks `clear_output()` which is method imported from `IPython` that clear output in running cell and `plot_trainers()` - that will plot all trainers that are currently stored in `selectioner`. Callbacks will be called in the same order as they were provided, so clear_output will erase output from previous trainer.


In [ ]:

def plot_trainters():
    src.vizualization.plot_trained_images(selectioner.trainers)
    plt.show()

callbacks = [clear_output, plot_trainters]

It is a core of generation process.  
- `train()` method will run training for every trainer. Trainers will apply gradient to image `epochs*steps` times.  
- After training history will be saved, so if selectioner removes trainer that is interesting for you, you will be able get back to them later.
- `sort_trainers_by_differences()` applies `ordering_method` to all pairs of trained images, and averages results per trainer. After that it sorts trainers in descending order, based on those averages. `sewar.full_ref.mse` (ordering method) measures similarity of images, so after sorting images that are the most different from all other will be on the top, and the least on the bottom.
- `remove_second_half_trainers()` does what it says. In this case removes trainers that are the most similar to each other.

In [ ]:

selectioner.train(epochs=1, steps=30, callbacks=callbacks)
selectioner.save_history()
selectioner.sort_trainers_by_differences(sewar.full_ref.mse)
clear_output()
src.vizualization.plot_trained_images(selectioner.trainers)
selectioner.remove_second_half_trainers()



Procedure defined above will be repeated two times in cells bellow.

In [ ]:

selectioner.train(epochs=1, steps=30, callbacks=callbacks)
selectioner.save_history()
selectioner.sort_trainers_by_differences(sewar.full_ref.mse)
clear_output()
src.vizualization.plot_trained_images(selectioner.trainers)
selectioner.remove_second_half_trainers()



After last selection process only four trainers remains. If you are happy with the results, you can save one of the trainers generations few cells below. If not, before "saving cell" there are some that allow you to train specified trainer longer.

In [ ]:
selectioner.train(epochs=1, steps=30, callbacks=callbacks)
selectioner.save_history()
selectioner.sort_trainers_by_differences(sewar.full_ref.mse)
clear_output()
src.vizualization.plot_trained_images(selectioner.trainers)


In [ ]:
assert False, "Exectution stopped on this cell, to allow you manually train selected images."

Definition of callbacks, similar to selectioner ones, but designed to work with single trainer.

In [ ]:
def display_output_image():
    display(trainer.output_image) 
trainer_callbacks = [clear_output, display_output_image]

You can pick one of the trainers (in this example third one), and train it little bit longer. After every epoch callbacks will be called.

In [ ]:
trainer = selectioner.trainers[2]
trainer.training_loop(epochs=4, steps_per_epoch=30, callbacks=trainer_callbacks)

`plot_trainer()` will produce plot that summarize trainer - it contains:
- Style and content images that were used to define loss function.
- Output image that trainer generated.
- Model schema with layers used in training marked on it. Styles layers marked on blue and content layers marked on red (if content layer is also style layer, then it has blue interior with red border)

In [ ]:
src.vizualization.plot_trainer(selectioner.trainers[2])

`save_vizualizations()` will save two images:
- raw generated image that will be stored in `./images/results` folder
- output generated by `plot_trainer()` function stored in `./images/trainers`  

Both images share the same generated name, that is combination of style and content images names. If there is already image with this name stored, then unique postfix will be added to new name.

In [ ]:
src.vizualization.save_vizualizations(
    selectioner.trainers[3],
    style_image_path,
    content_image_path,
    BASE_RESULT_PATH,
    BASE_TRAINER_PATH
)


If you want to come back to any of removed trainers you can do that uncommenting this cell. `selectioner.history[0][5]` will return six selectioner from first save.  
Similarly if you want to pick fourth selectioner from second save you should change this line to `selectioner.history[1][3]`

In [ ]:
################ HISTORY TRAINING #######################
# You can uncomment that if you want return to some trainers discarded in selection process.

# trainer = selectioner.history[0][5]
# trainer.training_loop(30, 4, callbacks=trainer_callbacks)

# src.vizualization.save_vizualizations(
#     trainer,
#     style_image_path,
#     content_image_path,
#     BASE_RESULT_PATH,
#     BASE_TRAINER_PATH
# )

### Models layers

In [ ]:
src.vizualization.plot_trainer(selectioner.trainers[0])
selectioner.trainers[0].style_layers, selectioner.trainers[0].content_layers

In [ ]:
src.vizualization.plot_trainer(selectioner.trainers[1])
selectioner.trainers[1].style_layers, selectioner.trainers[1].content_layers

In [ ]:
src.vizualization.plot_trainer(selectioner.trainers[2])
selectioner.trainers[2].style_layers, selectioner.trainers[2].content_layers

In [ ]:
src.vizualization.plot_trainer(selectioner.trainers[3])
selectioner.trainers[3].style_layers, selectioner.trainers[3].content_layers